In [1]:
import quartz

/home/jinjun/miniconda3/envs/quartz/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

gate_set = ['h', 'cx', 't', 'tdg']
ecc_file = 'bfs_verified_simplified.json'
no_increase = False
include_nop = True
quartz_context = quartz.QuartzContext(
    gate_set=gate_set,
    filename=ecc_file,
    no_increase=no_increase,
    include_nop=include_nop,
)
num_xfers = quartz_context.num_xfers
parser = quartz.PyQASMParser(context=quartz_context)
num_xfers

3904

In [3]:
with open('outputs/naive/out_graphs/1/0_58_0_0_s_0.000_0.000.qasm') as f:
    qasm_str = f.read()
qasm_str

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\nh q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[2],q[4];\nt q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[2],q[4];\ncx q[2],q[3];\nt q[4];\ntdg q[3];\ncx q[2],q[3];\nt q[2];\nt q[3];\nh q[3];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\nt q[3];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\ncx q[0],q[1];\nt q[3];\ntdg q[1];\nh q[3];\ncx q[0],q[1];\ncx q[3],q[4];\nt q[0];\nt q[1];\nt q[4];\ncx q[2],q[4];\ntdg q[4];\ncx q[3],q[4];\nt q[4];\ncx q[2],q[4];\ncx q[2],q[3];\ntdg q[4];\nt q[3];\nh q[4];\ncx q[2],q[3];\ntdg q[2];\ntdg q[3];\nh q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ntdg q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ncx q[0],q[1];\ntdg q[3];\nt q[1];\nh q[3];\ncx q[0],q[1];\ntdg q[0];\ntdg q[1];\n\n'

In [4]:
dag = parser.load_qasm_str(qasm_str)
graph = quartz.PyGraph(context=quartz_context, dag=dag)

Unknown gate: 


In [5]:
import time
s_time = time.time_ns()
avxfers_ref = []
for i in range(graph.num_nodes):
    avxfers_ref.append(
        graph.available_xfers(
            context=quartz_context,
            node=graph.get_node_from_id(id=i)
        )
    )
avxfers_ref
e_time = time.time_ns()
print(f'duration: { (e_time - s_time) / 1e6 } ms')
print(f'duration per node: { (e_time - s_time) / 1e6 / graph.num_nodes } ms')

duration: 1422.786023 ms
duration per node: 24.530793499999998 ms


In [55]:
import os
os.environ['OMP_SCHEDULE'] = 'dynamic'
for t in range(1000):
    s_time = time.time_ns()
    avxfers_para = []
    for i in range(graph.num_nodes):
        avxfers_para.append(
            graph.available_xfers_parallel(
                context=quartz_context,
                node=graph.get_node_from_id(id=i)
            )
        )
    avxfers_para
    e_time = time.time_ns()
    print(f'duration: { (e_time - s_time) / 1e6 }')
    assert len(avxfers_para) == len(avxfers_ref)
    for i in range(len(avxfers_para)):
        if avxfers_ref[i] != avxfers_para[i]:
            print(f'diff at {i}')
            ic(avxfers_ref[i])
            ic(avxfers_para[i])
            assert False
    print(f'succeed at {t}')

duration: 214.699639
succeed at 0
duration: 263.02931
succeed at 1


ic| avxfers_ref[i]: [438,
                     482,
                     483,
                     484,
                     485,
                     486,
                     487,
                     1245,
                     1336,
                     1403,
                     1404,
                     1405,
                     1406,
                     1407,
                     1408,
                     1665,
                     1666,
                     1667,
                     1668,
                     1669,
                     1670,
                     3649,
                     3775,
                     3903]
ic| avxfers_para[i]: [438,
                      482,
                      483,
                      484,
                      485,
                      487,
                      1245,
                      1336,
                      1403,
                      1404,
                      1405,
                      1406,
                      1408,
 

duration: 222.876529
diff at 7


AssertionError: 

In [20]:
from icecream import ic
assert len(avxfers_para) == len(avxfers_ref)
for i in range(len(avxfers_para)):
    if avxfers_ref[i] != avxfers_para[i]:
        print(f'diff at {i}')
        ic(avxfers_ref[i])
        ic(avxfers_para[i])
        assert False

In [47]:
t_start = time.time_ns()
x1 = graph.available_xfers(
    context=quartz_context,
    node=graph.get_node_from_id(id=7)
)
t_end = time.time_ns()
print(x1)
print(f'{(t_end - t_start) / 1e6}')

t_start = time.time_ns()
x2 = graph.available_xfers_parallel(
    context=quartz_context,
    node=graph.get_node_from_id(id=7)
)
t_end = time.time_ns()
print(x2)
print(f'{(t_end - t_start) / 1e6}')

[469, 470, 1275, 1276, 1479, 1480, 3444]
49.418834
[469, 470, 1275, 1276, 1479, 1480, 3444]
3.950958
